In [2]:
# Bible Verse Classification using Bidirectional LSTM and Pre-trained Embeddings

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
df = pd.read_csv("data/kjvdata.csv")

# Mapping book to author
author_list = {
    "Genesis": "Moses",
    "Exodus": "Moses",
    "Leviticus": "Moses",
    "Numbers": "Moses",
    "Deuteronomy": "Moses",
    "Joshua": "Joshua",
    "Judges": "Samuel, Nathan, Gad",
    "Ruth": "Samuel, Nathan, Gad",
    "1 Samuel (1 Kings)": "Samuel, Nathan, Gad",
    "2 Samuel (2 Kings)": "Samuel, Nathan, Gad",
    "1 Kings (3 Kings)": "Jeremiah",
    "2 Kings (4 Kings)": "Jeremiah",
    "1 Chronicles": "Ezra",
    "2 Chronicles": "Ezra",
    "Ezra": "Ezra",
    "Nehemiah": "Nehemiah, Ezra",
    "Esther": "Mordecai",
    "Job": "Job,Moses",
    "Psalms": "David,Asaph, Ezra, the sons of Korah, Heman, Ethan, Moses",
    "Proverbs": "Solomon ,Agur(30) and Lemuel(31)",
    "Ecclesiastes": "Solomon",
    "Song of Solomon (Canticles)": "Solomon",
    "Isaiah": "Isaiah",
    "Jeremiah": "Jeremiah",
    "Lamentations": "Jeremiah",
    "Ezekiel": "Ezekiel",
    "Daniel": "Daniel",
    "Hosea": "Hosea",
    "Joel": "Joel",
    "Amos": "Amos",
    "Obadiah": "Obadiah",
    "Jonah": "Jonah",
    "Micah": "Micah",
    "Nahum": "Nahum",
    "Habakkuk": "Habakkuk",
    "Zephaniah": "Zephaniah",
    "Haggai": "Haggai",
    "Zechariah": " Zechariah",
    "Malachi": "Malachi",
    "Matthew": "Matthew",
    "Mark": "John Mark",
    "Luke": "Luke",
    "John": "John, the Apostle",
    "Acts": "Luke",
    "Romans": "Paul",
    "1 Corinthians": "Paul",
    "2 Corinthians": "Paul",
    "Galatians": "Paul",
    "Ephesians": "Paul",
    "Philippians": "Paul",
    "Colossians": "Paul",
    "1 Thessalonians": "Paul",
    "2 Thessalonians": "Paul",
    "1 Timothy": "Paul",
    "2 Timothy": "Paul",
    "Titus": "Paul",
    "Philemon": "Paul",
    "Hebrews": "Paul, Luke, Barnabas, Apollos",
    "James": "James the brother of Jesus and Jude (not the Apostle, brother of John).",
    "1 Peter": "Peter",
    "2 Peter": "Peter",
    "1 John": "John, the Apostle",
    "2 John": "John, the Apostle",
    "3 John": "John, the Apostle",
    "Jude": "Jude, the brother of Jesus",
    "Revelation": "John, the Apostle"
}

df['author'] = df['book'].map(author_list)

# Prepare features and labels
x_features = df['text']
y_labels = df['author']

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_features)
sequences = tokenizer.texts_to_sequences(x_features)

# Padding sequences
max_sequence_length = 100
x_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Encode labels
encoder = LabelEncoder()
y_labels_enc = encoder.fit_transform(y_labels)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x_padded, y_labels_enc, test_size=0.33, random_state=42)

# Load GloVe embeddings
embedding_index = {}
with open("data/glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

# Create embedding matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Build Bidirectional LSTM model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False),
    Bidirectional(LSTM(128, return_sequences=False)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(encoder.classes_), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/20


/Users/skylercain/code/cs-6640-artificial-neural-networks/.direnv/python-3.11/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 97ms/step - accuracy: 0.1681 - loss: 2.9412 - val_accuracy: 0.3066 - val_loss: 2.3731
Epoch 2/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 49s 95ms/step - accuracy: 0.3063 - loss: 2.3415 - val_accuracy: 0.3779 - val_loss: 2.1394
Epoch 3/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 100ms/step - accuracy: 0.3732 - loss: 2.1140 - val_accuracy: 0.4026 - val_loss: 2.0424
Epoch 4/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 50s 96ms/step - accuracy: 0.4086 - loss: 1.9774 - val_accuracy: 0.4319 - val_loss: 1.9060
Epoch 5/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 52s 99ms/step - accuracy: 0.4483 - loss: 1.8578 - val_accuracy: 0.4477 - val_loss: 1.8910
Epoch 6/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 51s 97ms/step - accuracy: 0.4752 - loss: 1.7557 - val_accuracy: 0.4400 - val_loss: 1.8745
Epoch 7/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 53s 101ms/step - accuracy: 0.4958 - loss: 1.6762 - val_accuracy: 0.4640 - val_loss: 1.7642
Epoch 8/20
521/521 ━━━━━━━━━━━━━━━━━━━━ 51s 98ms/step - accuracy: 0.5218 - loss: 1.6064 - val_accur